#### Optimization model
##### Knapsack problem, Implementing Greedy algorithm

In [9]:
## Helper function for generating all power sets of a given list.
def power_set(input_set):
    n = len(input_set)
    # The number of subsets for a set of size n is 2^n
    total_subsets = 2**n

    # Generate all subsets using binary representation of numbers from 0 to 2^n - 1
    all_subsets = []
    for i in range(total_subsets):
        subset = [input_set[j] for j in range(n) if (i & (1 << j)) > 0]
        all_subsets.append(subset)

    return all_subsets

# Example usage:
input_set = [1, 9, 4]
result = power_set(input_set)
print(result)

[[], [1], [9], [1, 9], [4], [1, 4], [9, 4], [1, 9, 4]]


In [10]:
# Problem 2
# A house burglar wants to decide what items to take based on either the value of the items,
# the weights, or the ratio of value to weight
class Item(object):
    def __init__(self, n, v, w):
        self.name = n
        self.value = v
        self.weight = w

    def getName(self):
        return self.name
    def getValue(self):
        return self.value
    def getWeight(self):
        return self.weight
    def getDensity(self):
        return self.value/self.weight
    
    def __str__(self):
        return "<" + str(self.name) + ', ' + str(self.value) + ', ' + str(self.weight) + '>'
    
def value(item):
    return item.getValue()

def weightInverse(item):
    return 1/item.getWeight()

def density(item):
    return item.getValue()/item.getWeight()

def greedy(items, maxWeight, keyFunction):
    itemsCopy = sorted(items, key=keyFunction, reverse=True)
    totalValue, totalWeight = 0, 0
    result = []
    for i in range(len(itemsCopy)):
        if (totalWeight + itemsCopy[i].getWeight()) <= maxWeight:
            result.append(itemsCopy[i])
            totalValue += itemsCopy[i].getValue()
            totalWeight += itemsCopy[i].getWeight()
    
    return (result, totalValue)

def buildItems():
    names = ['clock','painting','radio','vase','book','computer']
    values = [175,90,20,50,10,200]
    weights = [10,9,4,2,1,20]
    Items = []
    for i in range(len(values)):
        Items.append(Item(names[i], values[i], weights[i]))
    return Items

###################################
# Implementing the greedy algorithm
###################################
# The essence of a greedy algorithm is making the best (as defined by some metric) 
# local choice at each step.

def testGreedy(items, maxWeight, keyFunction):
    taken, val = greedy(items, maxWeight, keyFunction)
    print('Total value of items taken is: ', val)
    for item in taken:
        print(' ', item)

def testGreedys(maxUnits):
    items = buildItems()
    print('Use greedy by value to fill a backpack of size: ', maxUnits)
    testGreedy(items, maxUnits, value)
    print('\nUse greedy by weight to fill a backpack with size: ', maxUnits)
    testGreedy(items, maxUnits, weightInverse)
    print('\nUse greedy by density to fill a backpack of size: ', maxUnits)
    testGreedy(items, maxUnits, density)

#############################################################
# Implementing brute force algorithm for the previous problem
#############################################################
# The complexity of the brute force algorithm is O(2^n)
# The brute force algorithm gives the optimal global solution,
# whereas the greedy algorithm gives the optimal local solution(based on a metric).

def chooseBest(pset, maxUnits):
    bestVal = 0
    bestSet = None
    for items in pset:
        itemsVal = 0
        itemsWeight = 0
        for item in items:
            itemsVal += item.getValue()
            itemsWeight += item.getWeight()
            if itemsWeight < maxUnits and itemsVal > bestVal:
                bestVal = itemsVal
                bestSet = items
    return (bestVal, bestSet)

def testBest(maxUnits = 20):
    items = buildItems()
    pset = power_set(items)
    val, taken = chooseBest(pset, maxUnits)
    print("Total value of items taken is: ", val)
    for item in taken:
        print(item)


In [11]:
testGreedys(20)
# Note: from the result of running this block we can see that the greedy solution doesn't always
# give the optimal solution and there can different criteria for finding the optimal solution.

Use greedy by value to fill a backpack of size:  20
Total value of items taken is:  200
  <computer, 200, 20>

Use greedy by weight to fill a backpack with size:  20
Total value of items taken is:  170
  <book, 10, 1>
  <vase, 50, 2>
  <radio, 20, 4>
  <painting, 90, 9>

Use greedy by density to fill a backpack of size:  20
Total value of items taken is:  255
  <vase, 50, 2>
  <clock, 175, 10>
  <book, 10, 1>
  <radio, 20, 4>


In [12]:
testBest(20)
# from the results of this block we can see how the brute force algorithm gives the optimal solution

Total value of items taken is:  265
<clock, 175, 10>
<painting, 90, 9>


### Decision tress and Dynamic Programming
Dynamic programming is a method for efficiently solving problems that exhibit the characteristics of overlapping subproblems and optimal substructure.

A problem has optimal substructure if a globally optimal solution can be found by combining optimal solutions to local subproblems.

A problem has optimal substructure if a globally optimal solution can be found by combining optimal solutions to local subproblems.

In [13]:
def fib(n):
    """Assumes n is an int >= 0
    Returns Fibonacci of n"""
    if n == 0 or n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

The previous implementation of Fibonacci algorithm is not efficient because for each n value, all of the previous values are calculated over and over again. For this reason dynamic programming offers a solution of recording the results of each run in a dictionary. This is called **Memoziation**.
The following code shows a new Fibonacci implementation that is more efficient. 

In [14]:
def fastFib(n, memo = {}):
    """Assumes n is an int >= 0, memo used only by recursive calls
    Returns Fibonacci of n"""
    if n == 0 or n == 1:
        return 1
    try:
        return memo[n]
    except KeyError:
        result = fastFib(n-1, memo) + fastFib(n-2, memo)
        memo[n] = result
        return result

fastFib(120)

The knapsack problem can be solved using dynamic programming by constructing search binary tree.
A rooted binary tree is an acyclic directed graph in which:
* There is exactly one node with no parent. This is called **Root**.
* Each non-root node has exactly one parent.
* Each node has at most two children nodes. A child-less node is called **leaf**.

Each node in the search tree for the 0/1 knapsack problem is labeled with a quadruple that denotes a partial solution to the knapsack problem. The elements of the quadruple are:
* A set of items to be taken,
* The list of items for which a decision has not been made,
* The total value of the items in the set of items to be taken (this is merely an optimization, since the value could be computed from the set), and
* The remaining space in the knapsack. (Again, this is an optimization, since it is merely the difference between the weight allowed and the weight of all the items taken so far.)

##### Implementation strategy:
The tree is built top-down starting with the root. One element is selected from the still-to-be-considered items. If there is room for that item in the knap- sack, a node is constructed that reflects the consequence of choosing to take that item. By convention, we draw that node as the left child. The right child shows the consequences of choosing not to take that item. The process is then applied recursively until either the knapsack is full or there are no more items to consider. Because each edge represents a decision (to take or not to take an item), such trees are called decision trees.

In [15]:
def maxVal(toConsider, avail):
    """Assumes toConsider a list of items, avail a weight
        Returns a tuple of the total value of a solution to the
        0/1 knapsack problem and the items of that solution"""
    if toConsider == [] or avail == 0:
        result = (0, ())
    elif toConsider[0].getWeight() > avail:
        # Explore right branch only
        result = maxVal(toConsider[1:], avail)
    else:
        nextItem = toConsider[0]
        # Explore left branch
        withVal, withTotake = maxVal(toConsider[1:], avail-nextItem.getWeight())
        withVal += nextItem.getValue()
        
        # Explore right branch
        withoutVal, withoutTotake = maxVal(toConsider[1:], avail)
        
        # Choose better branch
        if withVal > withoutVal:
            result = (withVal, withTotake + (nextItem,))
        else:
            result = (withoutVal, withoutTotake)
    return result

In [16]:
def testMaxVal(items, maxWeight):
    print("Use search tree to allocate {} weight".format(maxWeight))
    val, taken = maxVal(items, maxWeight)
    print("Total value of items taken: {}".format(val))
    for item in taken:
        print(' ', item)

x = buildItems()
testMaxVal(x, 30)

Use search tree to allocate 30 weight
Total value of items taken: 375
  <computer, 200, 20>
  <clock, 175, 10>


### Graph theory
* Set of nodes(vertices).
* Set of edges.
    * Undirected.
    * Directed.
    * Unweighted or weighted (weight can be any metric)
* Graphs are useful for capturing relationships between entities. 

* There are two representations of graphs:
    1. Adjacency matrix.
        * It can be used when the network is dense and we have a lot of edges.
        * Rows: source nodes.
        * Columns: destination nodes.
        * ex: c[s, d] = 1. (1 if there is a connection between the nodes s and d. 0 if otherwise.Additionally, we can use weight instead of 0s and 1s.)
    2. Adjacency list:
        * Associate with each node a list of destination nodes.
        

In [17]:
# Implementation
class Node(object):
    def __init__(self, name):
        """ Assumes name is a string """
        self.name = name
    def getName(self):
        return self.name
    def __str__(self):
        return self.name
    
class Edge(object):
    def __init__(self, src, dest):
        self.src = src
        self.dest = dest
    def getSource(self):
        return self.src
    def getDestination(self):
        return self.dest
    def __str__(self):
        return self.src.getName() + '-->' + self.dest.getName()
    
class Digraph(object):
    """ edges is a dictionary mapping each node to its children """
    def __init__(self):
        self.edges = {}

    def addNode(self, node):
        if node in self.edges:
            raise ValueError('Duplicate node')
        else:
            self.edges[node] = []
            
    def addEdge(self, edge):
        src = edge.getSource()
        dest = edge.getDestination()
        if not (src in self.edges and dest in self.edges):
            raise ValueError('Node NOT in graph!!')
        self.edges[src].append(dest)

    def children(self, node):
        return self.edges[node]
    def hasNode(self, node):
        return node in self.edges
    def getNode(self, name):
        for n in self.edges:
            if n.getName() == name:
                return n
        raise NameError(name)
            
    def __str__(self):
        result = ''
        for src in self.edges:
            for dest in self.edges[src]:
                result = result + src.getName() + ' --> ' + dest.getName() + '\n'
        return result[:-1]
    
class Graph(Digraph):
    def addEdge(self, edge):
        Digraph.addEdge(self, edge)
        rev = Edge(edge.getDestination(), edge.getSource())
        Digraph.addEdge(self, rev)

In [30]:
# Example
def buildcitygraph(graphType):
    g = graphType()
    cities = ('Boston', 'Providence', 'New York', 'Chicago', 'Denver', 'Phoenix', 'Los Angeles')
    for name in cities:
        g.addNode(Node(name))

    g.addEdge(Edge(g.getNode('Boston'), g.getNode('Providence')))
    g.addEdge(Edge(g.getNode('Boston'), g.getNode('New York')))
    g.addEdge(Edge(g.getNode('Providence'), g.getNode('Boston')))
    g.addEdge(Edge(g.getNode('New York'), g.getNode('Boston')))
    g.addEdge(Edge(g.getNode('New York'), g.getNode('Chicago')))
    g.addEdge(Edge(g.getNode('Chicago'), g.getNode('Denver')))
    g.addEdge(Edge(g.getNode('Denver'), g.getNode('Phoenix')))
    g.addEdge(Edge(g.getNode('Denver'), g.getNode('New York')))
    g.addEdge(Edge(g.getNode('Chicago'), g.getNode('Phoenix')))
    g.addEdge(Edge(g.getNode('Los Angeles'), g.getNode('Boston')))
    return g

print(buildcitygraph(Digraph))

Boston --> Providence
Boston --> New York
Providence --> Boston
New York --> Boston
New York --> Chicago
Chicago --> Denver
Chicago --> Phoenix
Denver --> Phoenix
Denver --> New York
Los Angeles --> Boston


In [33]:
# Finding the shortest path
# Depth First Search (DFS)

def printPath(path):
    result = ''
    for i in range(len(path)):
        result = result + str(path[i])
        if i != len(path) -1:
            result = result + ' -> '
    return result 

def DFS(graph, start, end, path, shortest, toPrint=False):
    path = path + [start]

    if toPrint:
        print('Current DFS path', printPath(path))
        
    if start == end:
        return path
    for node in graph.children(start):
        if node not in path:
            if shortest == None or len(path) < len(shortest):
                newPath = DFS(graph, node, end, path, shortest, toPrint)

                if newPath != None:
                    shortest = newPath
        elif toPrint:
            print('Already visited', node)
    return shortest

def shortestPath(graph, start, end, toPrint):
    return DFS(graph, start, end, [], None, toPrint)

def testSP(source, destination):
    g = buildcitygraph(Digraph)
    sp = shortestPath(g, g.getNode(source), g.getNode(destination), toPrint = True)
    if sp != None:
        print('Shortest path from {} to {} is: {}'.format(source, destination, printPath(sp)))
    else:
        print('There is no path from {} to {}'.format(source, destination))

testSP('Boston', 'Phoenix')

Current DFS path Boston
Current DFS path Boston -> Providence
Already visited Boston
Current DFS path Boston -> New York
Already visited Boston
Current DFS path Boston -> New York -> Chicago
Current DFS path Boston -> New York -> Chicago -> Denver
Current DFS path Boston -> New York -> Chicago -> Denver -> Phoenix
Already visited New York
Current DFS path Boston -> New York -> Chicago -> Phoenix
Shortest path from Boston to Phoenix is: Boston -> New York -> Chicago -> Phoenix
